In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data_raw'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary 

./data_raw/movie_conversations.tsv
./data_raw/movie_titles_metadata.tsv
./data_raw/movie_characters_metadata.tsv
./data_raw/movie_lines.tsv
./data_raw/README.txt
./data_raw/raw_script_urls.tsv


In [120]:
convo_data = pd.read_csv('./data_raw/movie_conversations.tsv', sep="\t", header=None, names=["first_char", "second_char", "movie_id", "convo"])

cols = [f'col_{i}' for i in range(0,150)]
lines_data = pd.read_csv('./data_raw/movie_lines.tsv', sep="\t", names=cols)
# Rename columns
lines_data.rename(columns={'col_0': 'line_id', 'col_1': 'character_id', 'col_2': 'movie_id', 'col_3': 'character_name', 'col_4': 'text'}, inplace=True)
lines_data.drop(list(lines_data)[5:], axis=1, inplace=True)

convo_data.head(), lines_data.head()

/var/folders/6c/81hr3gfj4rs2c3wnmbn0xc1m0000gn/T/ipykernel_52849/3897709760.py:4: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,25,26,27,28,31,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  lines_data = pd.read_csv('./data_raw/movie_lines.tsv', sep="\t", names=cols)


(  first_char second_char movie_id                          convo
 0         u0          u2       m0  ['L194' 'L195' 'L196' 'L197']
 1         u0          u2       m0                ['L198' 'L199']
 2         u0          u2       m0  ['L200' 'L201' 'L202' 'L203']
 3         u0          u2       m0         ['L204' 'L205' 'L206']
 4         u0          u2       m0                ['L207' 'L208'],
   line_id character_id movie_id character_name          text
 0   L1045           u0       m0         BIANCA  They do not!
 1   L1044           u2       m0        CAMERON   They do to!
 2    L985           u0       m0         BIANCA    I hope so.
 3    L984           u2       m0        CAMERON     She okay?
 4    L925           u0       m0         BIANCA     Let's go.)

In [121]:
# Get conversations in string format from primary keys
for i, dat in enumerate(convo_data['convo']):
  # Convert list literal to actual list
  dat = [i.strip("'") for i in dat.strip('][').split(' ')]
  convo_data.loc[i]['convo'] = dat

spidy_convo = convo_data.loc[convo_data['movie_id'] == 'm189']
spidy_convo['response'] = None
spidy_convo.head()

/var/folders/6c/81hr3gfj4rs2c3wnmbn0xc1m0000gn/T/ipykernel_52849/757153175.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spidy_convo['response'] = None


,first_char,second_char,movie_id,convo,response
26475,u2863,u2871,m189,"[L529861, L529862]",None
26476,u2863,u2871,m189,"[L529866, L529867]",None
26477,u2863,u2871,m189,"[L530010, L530011]",None
26478,u2863,u2871,m189,"[L530016, L530017]",None
26479,u2863,u2871,m189,"[L530024, L530025]",None


In [122]:
for indx, data in enumerate(spidy_convo.iterrows()):
  data = data[1]
  conv = [": ".join([lines_data.loc[lines_data['line_id'] == i]['character_name'].values[0], lines_data.loc[lines_data['line_id'] == i]['text'].values[0]]) for i in data['convo']]
  # print('LIZ' in str(conv))
  # data['convo']  = "\n".join(conv)
  if ('PETER:' in str(conv)) or ('SPIDER-MAN:' in str(conv)):
    idx_from_end=0
    res = ""
    if "PETER:" in str(conv):
      idx_from_end = max([i if 'PETER:' in st else 0 for i, st in enumerate(conv)])
      res = conv[idx_from_end][7:]
      conv[idx_from_end] =  "PETER: <RESPONSE-HERE>"
    else:
      idx_from_end = max([i if 'SPIDER-MAN:' in st else 0 for i, st in enumerate(conv)])
      res = conv[idx_from_end][11:]
      conv[idx_from_end] = "SPIDER-MAN: <RESPONSE-HERE>"

    spidy_convo.iloc[indx]['convo'] = "\n".join(conv)
    spidy_convo.iloc[indx]['response'] = res


In [135]:
finished = spidy_convo.loc[spidy_convo['response'].notnull()][['convo', 'response']]

In [136]:
finished

,convo,response
26480,BEN: Here you go Pete.\nPETER: Uh Uncle Ben I-...,No. No that's okay.
26481,BEN: So uh... how's college goin'?\nPETER: Sam...,I know. Still thinking about retiring?
26482,BEN: Ya still follow the Mets Pete?\nPETER: No...,Remember Little League?
26483,BEN: Ya set for uh ya know-- money?\nPETER: Oh...,Oh come on Ben that's not--
26484,BEN: When you won that scholarship I was proud...,I know.
...,...,...
26590,ROZ: I listened to him talk of eternal truth a...,Just drop me off here.
26591,ROZ: Do you... live around here?\nSPIDER-MAN: ...,Phooey on that.
26592,ROZ: Spider-Man be careful! He's quite mad.\n...,I'm not so happy myself.
26593,SPIDER-MAN: <RESPONSE-HERE>\nROZ: Sure in the ...,Professor you ever fly one of these things be...


In [137]:
# Write to jsonl file
finished.to_json('./spidy_convo.jsonl', orient='records', lines=True)